In [1]:
import os
import pandas as pd
import swat
import sys
import fnmatch

## CONNECTION TO Cloud Analytic Services and activation of necessary CAS Actions sets 

In [2]:
cashost='sepviya35.aws.sas.com'
casport=5570

sess = swat.CAS(cashost, casport)
sess.loadactionset('fedsql')
sess.loadactionset('datastep')

NOTE: Added action set 'fedsql'.
NOTE: Added action set 'datastep'.


[actionset]

 'datastep'

+ Elapsed: 0.000272s, user: 0.000148s, sys: 7.8e-05s, mem: 0.221mb

## STEP 0 : Get the list of truck data files located in a given directory

In [3]:
filling_data_path="/opt/poc/Poclain/Phase1_Clean/"
csvfilelist=fnmatch.filter(os.listdir(path=filling_data_path), '*.csv')
csvfilelist.sort()

#csvfilelist

# I have launch this command to cleanse a specific file from uncomplete records : awk -F'\t' 'NF==28 {print}' infile  > newfile

## Define function in charge of importing data

In [4]:
def import_filling_data(filename, castabname):
    print('Uploading file : '+filename+' into cas in-memory table : '+castabname)
    sess.upload_file(filling_data_path+filename, 
                 casout=castabname, 
                 importoptions={"fileType":"csv","getNames":True, "guessRows":100,"delimiter":";"})

## Define function in charge of appending a table to another in CAS

In [6]:
def append_filling_data(targetTable, incrementTable):
    appendcode="data '"+targetTable+"'(APPEND=YES); set '"+incrementTable+"'; run;"
    sess.dataStep.runCode(appendcode)

## Drop exiting final table

In [8]:
sess.droptable(name="TRUCK_FULL", caslib="CASUSER", quiet="true")
sess.droptable(name="TRUCK_FULL", caslib="PUBLIC", quiet="true")
sess.droptable(name="TRUCK_FULL_CLEAN", caslib="PUBLIC", quiet="true")

+ Elapsed: 0.00028s, user: 0.000231s, mem: 0.291mb

## IMPORT FIRST DATA FILE

In [9]:
import_filling_data(csvfilelist[0], "TRUCK_FULL")

Uploading file : PAUL-DAF-TRUCK_2019-01-14T08-24-20.554Z.csv into cas in-memory table : TRUCK_FULL
NOTE: Cloud Analytic Services made the uploaded file available as table TRUCK_FULL in caslib CASUSER(viyademo01).
NOTE: The table TRUCK_FULL has been created in caslib CASUSER(viyademo01) from binary data uploaded to Cloud Analytic Services.


## IMPORT ALL REMAINING DATA FILES

In [10]:
for i in range(1,len(csvfilelist)):
    sess.droptable(table="TMP_TRUCK", quiet="true")
    import_filling_data(csvfilelist[i], "TMP_TRUCK")
    append_filling_data("TRUCK_FULL", "TMP_TRUCK")

Uploading file : PAUL-DAF-TRUCK_2019-01-14T09-36-19.544Z.csv into cas in-memory table : TMP_TRUCK
NOTE: Cloud Analytic Services made the uploaded file available as table TMP_TRUCK in caslib CASUSER(viyademo01).
NOTE: The table TMP_TRUCK has been created in caslib CASUSER(viyademo01) from binary data uploaded to Cloud Analytic Services.
Uploading file : PAUL-DAF-TRUCK_2019-01-14T09-36-56.304Z.csv into cas in-memory table : TMP_TRUCK
NOTE: Cloud Analytic Services made the uploaded file available as table TMP_TRUCK in caslib CASUSER(viyademo01).
NOTE: The table TMP_TRUCK has been created in caslib CASUSER(viyademo01) from binary data uploaded to Cloud Analytic Services.
Uploading file : PAUL-DAF-TRUCK_2019-01-14T09-41-06.555Z.csv into cas in-memory table : TMP_TRUCK
NOTE: Cloud Analytic Services made the uploaded file available as table TMP_TRUCK in caslib CASUSER(viyademo01).
NOTE: The table TMP_TRUCK has been created in caslib CASUSER(viyademo01) from binary data uploaded to Cloud Analy

## Promote the full in-memory table for other users

In [11]:
sess.promote(name="TRUCK_FULL", targetCaslib="PUBLIC")

NOTE: Cloud Analytic Services promoted table TRUCK_FULL in caslib CASUSER(viyademo01) to table TRUCK_FULL in caslib PUBLIC.


+ Elapsed: 0.0205s, sys: 0.0202s, mem: 0.332mb

## Save the in-memory table on disk to secure the process

In [12]:
sess.table.save(table=dict(name='TRUCK_FULL', caslib='PUBLIC'), caslib='PUBLIC', name='TRUCK_FULL.sashdat', replace=True)

NOTE: Cloud Analytic Services saved the file TRUCK_FULL.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'TRUCK_FULL.sashdat'

+ Elapsed: 0.371s, user: 0.0308s, sys: 0.175s, mem: 19mb

In [13]:
sess.table.columninfo(table=dict(name='TRUCK_FULL', caslib='PUBLIC'))

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,Relative_Time_1Hz,,1,varchar,26,26,,0,0
1,Date_Time,,2,varchar,26,26,,0,0
2,Date,,3,varchar,9,9,,0,0
3,Day_Time,,4,varchar,8,8,,0,0
4,Immatriculation,,5,varchar,6,6,,0,0
...,...,...,...,...,...,...,...,...,...
102,LS_Percent_Pressure_Value,,103,double,8,12,,0,0
103,LS_Release_Valve_Y2,,104,double,8,12,,0,0
104,P_Release_Valve_Y3,,105,double,8,12,,0,0
105,Reserved,,106,double,8,12,,0,0


## Import fixed width file for Broyeur data

In [3]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

In [4]:
find("time          HPM_A1_Pressure_bar HPM_A2_Pressure_bar HPM_B1_Pressure_bar HPM_B2_Pressure_bar LP_A_Pressure_bar LP_B_Pressure_bar RLFF_C_Flow_lpm TMP_A_Temperature_deg TMP_B_Temperature_deg TMP_C_Temperature_deg TR_A_Speed_rpm TR_B_Speed_rpm Motors_A_Gyro_Pitch_rate Motors_A_Gyro_Roll_rate Motors_A_Gyro_Yaw_rate Motors_A_Gyro_Pitch Motors_A_Gyro_Roll Motors_A_Gyro_Accel_Y Motors_A_Gyro_Accel_X Motors_A_Gyro_Accel_Z Motors_B_Gyro_Pitch_rate Motors_B_Gyro_Roll_rate Motors_B_Gyro_Yaw_rate Motors_B_Gyro_Pitch Motors_B_Gyro_Roll Motors_B_Gyro_Accel_Y Motors_B_Gyro_Accel_X Motors_B_Gyro_Accel_Z Motors_C_Gyro_Pitch_rate Motors_C_Gyro_Roll_rate Motors_C_Gyro_Yaw_rate Motors_C_Gyro_Pitch Motors_C_Gyro_Roll Motors_C_Gyro_Accel_Y Motors_C_Gyro_Accel_X Motors_C_Gyro_Accel_Z Oil_Viscosity Oil_Density Oil_DieleectricConstant Oil_AverageTemperature Oil_ImpedBelowNorm_ShortedToGND Oil_ImpedAboveNorm_SensorDamage Oil_ASIC_OverTemp Oil_AlgoFailedToConverge Oil_RTD_ResistBelowNorm_ShortedToGND Oil_RTD_ResistAboveNorm_SensDamaged Oil_RTD_OverTempCond Oil_Internal_SystemError"," ")

[4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 33,
 53,
 73,
 93,
 111,
 129,
 145,
 167,
 189,
 211,
 226,
 241,
 266,
 290,
 313,
 333,
 352,
 374,
 396,
 418,
 443,
 467,
 490,
 510,
 529,
 551,
 573,
 595,
 620,
 644,
 667,
 687,
 706,
 728,
 750,
 772,
 786,
 798,
 822,
 845,
 877,
 909,
 927,
 952,
 989,
 1025,
 1046]

In [5]:
%%time

# The command :  awk '{if ((NR!=3) && (NR!=1)) {print}}' extract-Broyeur.csv > extract-Broyeur-clean.csv as been executed to remove second line with dash

broyeur_castab=sess.CASTable("broyeur_raw")

colspecs=[
[0,13],[14,33],[34,53],[54,73],[74,93],[94,111],[112,129],[130,145],[146,167],[168,189],
[190,211],[212,226],[227,241],[242,266],[267,290],[291,313],[314,333],[334,352],
[353,374],[375,396],[397,418],[419,443],[444,467],[468,490],[491,510],[511,529],
[530,551],[552,573],[574,595],[596,620],[621,644],[645,667],[668,687],
[688,706],[707,728],[729,750],[751,772],[773,786],[787,798],[799,822],[823,845],
[846,877],[878,909],[910,927],[928,952],[953,989],[990,1025],
[1026,1046],[1047,1071]]

names=['time','HPM_A1_Pressure_bar','HPM_A2_Pressure_bar','HPM_B1_Pressure_bar','HPM_B2_Pressure_bar','LP_A_Pressure_bar','LP_B_Pressure_bar','RLFF_C_Flow_lpm',
'TMP_A_Temperature_deg','TMP_B_Temperature_deg','TMP_C_Temperature_deg','TR_A_Speed_rpm',
'TR_B_Speed_rpm','Motors_A_Gyro_Pitch_rate','Motors_A_Gyro_Roll_rate','Motors_A_Gyro_Yaw_rate',
'Motors_A_Gyro_Pitch','Motors_A_Gyro_Roll','Motors_A_Gyro_Accel_Y','Motors_A_Gyro_Accel_X',
'Motors_A_Gyro_Accel_Z','Motors_B_Gyro_Pitch_rate','Motors_B_Gyro_Roll_rate','Motors_B_Gyro_Yaw_rate',
'Motors_B_Gyro_Pitch','Motors_B_Gyro_Roll','Motors_B_Gyro_Accel_Y','Motors_B_Gyro_Accel_X','Motors_B_Gyro_Accel_Z',
'Motors_C_Gyro_Pitch_rate','Motors_C_Gyro_Roll_rate','Motors_C_Gyro_Yaw_rate','Motors_C_Gyro_Pitch','Motors_C_Gyro_Roll',
'Motors_C_Gyro_Accel_Y','Motors_C_Gyro_Accel_X','Motors_C_Gyro_Accel_Z','Oil_Viscosity','Oil_Density','Oil_DieleectricConstant',
'Oil_AverageTemperature','Oil_ImpedBelowNorm_ShortedToGND','Oil_ImpedAboveNorm_SensorDamage','Oil_ASIC_OverTemp','Oil_AlgoFailedToConverge',
'Oil_RTD_ResistBelowNorm_ShortedToGND','Oil_RTD_ResistAboveNorm_SensDamaged','Oil_RTD_OverTempCond','Oil_Internal_SystemError']

sess.read_fwf('/opt/poc/Poclain/broyeur/extract-Broyeur-clean-noname.csv',casout="broyeur_raw",colspecs=colspecs,names=names)

NOTE: Cloud Analytic Services made the uploaded file available as table BROYEUR_RAW in caslib CASUSER(viyademo01).
NOTE: The table BROYEUR_RAW has been created in caslib CASUSER(viyademo01) from binary data uploaded to Cloud Analytic Services.
CPU times: user 2min 20s, sys: 3.51 s, total: 2min 23s
Wall time: 2min 24s


CASTable('BROYEUR_RAW', caslib='CASUSER(viyademo01)')

In [8]:
sess.droptable(name="broyeur_raw", Caslib="DNFS", quiet=True)

+ Elapsed: 0.000363s, sys: 0.000986s, mem: 0.291mb

In [9]:
sess.promote(name="broyeur_raw", targetCaslib="DNFS")

In [10]:
sess.table.save(table=dict(name='broyeur_raw', caslib='DNFS'), caslib='DNFS', name='broyeur.parquet', replace=True)

NOTE: Cloud Analytic Services saved the file broyeur.parquet in caslib DNFS.


[caslib]

 'DNFS'

[name]

 'broyeur.parquet'

+ Elapsed: 1.73s, user: 6.36s, sys: 0.492s, mem: 909mb

In [11]:
sess.terminate()